# Prep

In [3]:
import pandas as pd
import numpy as np
import pprint

## Functions

### Extract

In [4]:
def get_raw_workouts(fp):
  all_workouts = []
  counter = 0

  with open(fp, encoding='utf-8') as f:
    clean_strings = [line.strip() for line in f.readlines()]
    no_empty_lines = list(filter(None, clean_strings))
    no_empty_lines.pop(0) # tirar a primeira linha, que é vazia

    single_workout = []
    for line in no_empty_lines:
      if 'Week' in line:
        if counter > 0:
          all_workouts.append(single_workout)
          single_workout = []
        counter += 1
      single_workout.append(line)

  return all_workouts

### Transform

In [10]:
def get_workout_dict(raw_workout_list):
    workout_listdict = []

    for workout in raw_workout_list:
        workout_description = workout[0].split(';')

        workout_dict = {
            #'raw_description': f'{workout[0]}', #só pra conferir por enquanto, retirar depois
            'description': {
                'info': workout_description[0],
                'start_time': workout_description[1],
                'duration': workout_description[2]
            },
            'exercises': []
        }

        current_exercise = {'exercise': '', 'data': []}
        for line in workout[1:]:
            if len(line) > 13:
                if len(current_exercise['exercise']) > 0:
                    workout_dict['exercises'].append(current_exercise)
                current_exercise = {'exercise': line, 'data': []}
            else:
                current_exercise['data'].append(line)
        workout_dict['exercises'].append(current_exercise) # necessary to append the last dict

        workout_listdict.append(workout_dict)
    
    return workout_listdict

# Extract

In [11]:
raw_workouts = get_raw_workouts(r'G:\My Drive\Projetos\workouts\2024_08_25 Workouts.csv')
raw_workouts.reverse()

# Transform

In [12]:
workouts_dict = get_workout_dict(raw_workouts)

In [13]:
pprint.pprint(workouts_dict[12], sort_dicts=False)

{'description': {'info': '"Day 2 - Costas Bíceps Triceps · Day 2 · Week 1 · '
                         'Padrão2 · Deload"',
                 'start_time': '"2023-11-06 16:39 h"',
                 'duration': '"47 min"'},
 'exercises': [{'exercise': '"1. Lat Pulldowns with Wide Overhand Grip · Cable '
                            '· 15 reps"',
                'data': ['#;KG;REPS', '1;45;17', '2;45;17', '3;45;15']},
               {'exercise': '"2. Chest-Supported Low Rows with Neutral Grip · '
                            'Machine · 15 reps"',
                'data': ['#;KG;REPS', '1;80;15', '2;80;15', '3;80;15']},
               {'exercise': '"3. Triceps Pushdowns · Cable · 15 reps"',
                'data': ['#;KG;REPS', '1;25;17', '2;25;17', '3;25;15']},
               {'exercise': '"4. Curls · EZ bar · 15 reps"',
                'data': ['#;KG;REPS', '1;14;17', '2;14;17', '3;14;15']},
               {'exercise': '"5. Rows with Wide Neutral Grip · Cable · 15 '
                         